## Driver — run all panels

This notebook delegates all plotting and aggregation to `_panels_v0_cells.py`.
Inputs are auto-loaded from `../artifacts/*.json` if present (decision/paradox/trace);
otherwise safe defaults are created so panels degrade gracefully.


In [ ]:
# Driver: load env (../artifacts or safe defaults) and render all panels
from _panels_v0_cells import maybe_load_env, run_all_panels

maybe_load_env(globals())  # ensures runs_df / axes_df / trace_dashboard exist
run_all_panels(globals())   # render all dashboard panels
